In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16747,2024-03-09,Brasil Nbb,17:00,Pinheiros,Fortaleza B.C.,1.67,2.12,155.5,1.87,1.83,-2.5,1.87,2.40,hdvgOhjU,0.598802,0.471698,0.534759,0.546448,0.070501,0.6,0.4,NaN,NaN,159.692,67.147036,0.420478,147.588,64.768793,0.438849,147.602,290.686,269.01,237.60,0.0,0.0,0.0,0.0,0.167915,0.015289,0.175535,-1.88,-0.376,-1.781915,0.547739,0.5,-0.047739,-2.03,-0.406,-2.758621,0.348122,0.3,-0.048122
16748,2024-03-09,Eua Nba,00:00,Los Angeles Lakers,Milwaukee Bucks,2.39,1.62,228.5,1.81,1.95,2.5,1.97,1.36,dWZsXZqj,0.418410,0.617284,0.552486,0.512821,0.035694,0.4,0.0,NaN,NaN,228.290,51.791379,0.226867,203.204,46.914796,0.230875,233.270,191.880,206.40,207.90,0.0,0.0,0.0,0.0,0.271557,0.052657,0.259060,0.53,0.106,13.113208,0.604878,0.7,0.095122,0.46,0.092,6.739130,0.588754,0.4,-0.188754
16749,2024-03-09,Eua Nba,00:00,Portland Trail Blazers,Houston Rockets,2.92,1.44,218.5,1.93,1.93,4.5,2.05,1.20,zNYoWgad,0.342466,0.694444,0.518135,0.518135,0.036910,0.6,0.4,NaN,NaN,333.292,76.421451,0.229293,256.030,48.111647,0.187914,377.860,259.896,435.89,176.28,1.0,0.0,0.0,0.0,0.480054,0.000000,0.369871,0.04,0.008,240.000000,0.339235,0.5,0.160765,-5.00,-1.000,-0.440000,0.442994,0.2,-0.242994
16750,2024-03-09,Itália Liga A,16:30,Treviso,Reggiana,2.12,1.73,161.5,1.88,1.92,1.5,1.95,1.59,faqh4jXu,0.471698,0.578035,0.531915,0.520833,0.049733,0.4,0.6,NaN,NaN,236.480,53.005620,0.224144,180.866,71.726924,0.396575,177.560,196.848,280.60,300.51,0.0,0.0,0.0,0.0,0.143258,0.014886,0.143818,-0.97,-0.194,-5.773196,0.000000,0.0,0.000000,-2.50,-0.500,-1.460000,0.000000,0.0,0.000000
16751,2024-03-09,Alemanha Bbl,14:30,Oldenburg,Wurzburg,1.78,2.04,165.5,1.81,1.95,-2.5,2.03,2.33,rsHT91Ir,0.561798,0.490196,0.552486,0.512821,0.051994,0.8,0.8,NaN,NaN,248.676,140.052468,0.563193,176.224,55.942055,0.317449,171.808,201.102,91.20,272.60,0.0,0.0,0.0,0.0,0.096255,0.052657,0.097308,-0.35,-0.070,-11.142857,0.000000,0.0,0.000000,1.16,0.232,4.482759,0.496363,0.7,0.203637
16752,2024-03-09,Alemanha Bbl,16:00,Tubingen,Syntainics MBC,1.78,2.04,181.5,1.80,1.99,-2.5,2.01,2.33,fyeyTtPK,0.561798,0.490196,0.555556,0.502513,0.051994,0.6,0.4,NaN,NaN,351.170,318.809492,0.907849,313.900,158.075858,0.503587,257.224,291.286,381.30,273.78,1.0,0.0,0.0,0.0,0.096255,0.070897,0.104274,-5.00,-1.000,-0.780000,0.368482,0.2,-0.168482,3.89,0.778,1.336761,0.000000,0.0,0.000000
16753,2024-03-09,Alemanha Pro A,15:30,Paderborn,Vechta 2,2.07,1.70,165.5,1.83,1.87,1.5,1.92,1.56,jgqmmKM3,0.483092,0.588235,0.546448,0.534759,0.071327,0.8,0.4,NaN,NaN,319.274,172.795664,0.541214,213.448,81.690102,0.382717,288.358,312.422,428.73,327.63,1.0,0.0,0.0,0.0,0.138795,0.015289,0.146298,-0.44,-0.088,-12.159091,0.000000,0.0,0.000000,-0.63,-0.126,-5.555556,0.000000,0.0,0.000000
16754,2024-03-09,Áustria Liga Zweite,14:00,Gussing Blackbirds,Worthersee Piraten,1.97,1.76,136.5,1.82,1.88,-1.5,2.03,1.87,6Fbhm8xO,0.507614,0.568182,0.549451,0.531915,0.075796,0.0,0.4,NaN,NaN,167.890,77.106512,0.459268,134.502,39.570743,0.294202,0.000,132.674,96.38,184.08,0.0,0.0,0.0,0.0,0.079621,0.022933,0.058019,0.00,0.000,inf,0.000000,0.0,0.000000,-1.48,-0.296,-2.567568,0.000000,0.0,0.000000
16755,2024-03-09,Bósnia E Herzegovina Prvenstvo Bih,14:00,Promo,Orlovik,1.74,1.99,162.5,1.86,1.80,-2.5,2.01,2.39,rVrlPOCh,0.574713,0.502513,0.537634,0.555556,0.077225,0.6,0.6,NaN,NaN,273.976,160.617062,0.586245,267.376,163.208633,0.610409,271.612,339.574,116.64,308.70,0.0,0.0,0.0,0.0,0.094786,0.023184,0.122

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:00,Brasil Nbb,Pinheiros,Fortaleza B.C.,155.5,1.87,1.0000,Over
1,14:30,Alemanha Bbl,Oldenburg,Wurzburg,165.5,1.81,1.0000,Over
2,16:00,Alemanha Bbl,Tubingen,Syntainics MBC,181.5,1.80,1.0000,Over
3,14:00,Áustria Liga Zweite,Gussing Blackbirds,Worthersee Piraten,136.5,1.82,0.6124,Over
4,14:00,Bósnia E Herzegovina Prvenstvo Bih,Promo,Orlovik,162.5,1.86,1.0000,Over
5,05:00,Coreia Do Sul Kbl,Seoul Thunders,Goyang Sono,164.5,1.83,1.0000,Over
6,16:15,Croácia Premijer Liga,GKK Sibenka,Dinamo Zagreb,160.5,1.86,1.0000,Over
7,14:00,Eslováquia Extraliga,BKM Lucenec,Inter Bratislava,165.5,1.87,1.0000,Over
8,00:00,Eua Ncaa,San Diego State,Boise State,135.5,1.91,1.0000,Over
9,12:00,França Lnb,Le Mans,Nanterre,161.5,1.80,1.0000,Over
